# Biol 359A | Write equations for diagrams
### Spring 2025, Week 7
Objectives:
- Convert diagrams to equations
- Simulate signal transduction systems with differential equations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, interactive, fixed, FloatSlider, HBox, VBox, Layout, Button
from IPython.display import display, clear_output

In [ ]:
! rm -r week7_diagramToEquation/
! git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr25/week7_diagramToEquation.git
! cp -r week7_diagramToEquation/* .
! ls

## Write equations for diagrams - Signal transduction systems (step function)

In [ ]:
from IPython import display
display.Image("images/signal_transduction.png", width=600)

### Define system of ODEs

In [ ]:
def solve_signal_transduction_odes(
    t_max=10.0, 
    X_init=28, XE_init=0.1, XP_init=0.2, XPE_init=0.01, XPP_init=0.02,
    E_init=0.01, E_step=10,
    k1=1, k2=0.1, k3=4, k4=1, k5=0.1, k6=4, k7=2, k8=2
):
    """
    Solves the system of ODEs for the signal transduction model
    """
    # Define the ODE system
    def ode_system(t, y):
        X, XE, XP, XPE, XPP = y
        
        # Handle the step change in E at t = 1
        E_current = E_step if t >= 1 else E_init
        
        # System of ODEs
        dXdt = -k1 * X * E_current + k2 * XE + k7 * XP
        dXEdt = k1 * X * E_current - (k2 + k3) * XE
        dXPdt = k3 * XE - k7 * XP - k4 * XP * E_current + k5 * XPE + k8 * XPP
        dXPEdt = k4 * XP * E_current - (k5 + k6) * XPE
        dXPPdt = k6 * XPE - k8 * XPP
        
        return [dXdt, dXEdt, dXPdt, dXPEdt, dXPPdt]
    
    # Initial conditions
    y0 = [X_init, XE_init, XP_init, XPE_init, XPP_init]
    
    # Time points
    t_span = (0, t_max)
    t_eval = np.linspace(0, t_max, 1000)
    
    # Solve the ODE system
    solution = solve_ivp(
        ode_system, 
        t_span, 
        y0, 
        method='RK45', 
        t_eval=t_eval,
        rtol=1e-6,
        atol=1e-8
    )
    
    return solution

### Plot functions

In [ ]:
def plot_ode_solution(
    t_max=10.0, 
    X_init=28, XE_init=0.1, XP_init=0.2, XPE_init=0.01, XPP_init=0.02,
    E_init=0.01, E_step=10,
    k1=1, k2=0.1, k3=4, k4=1, k5=0.1, k6=4, k7=2, k8=2
):
    """
    Solves the ODE system and plots the results
    """
    # Solve the system
    solution = solve_signal_transduction_odes(
        t_max, X_init, XE_init, XP_init, XPE_init, XPP_init,
        E_init, E_step, k1, k2, k3, k4, k5, k6, k7, k8
    )
    
    t = solution.t
    X, XE, XP, XPE, XPP = solution.y
    
    # Create E array with step change at t=1
    E = np.ones_like(t) * E_init
    E[t >= 1] = E_step
    
    # Create a figure with a specific size
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot the results
    ax[0].plot(t, X, label='X', linewidth=2)
    ax[0].plot(t, XE, label='XE', linewidth=2)
    ax[0].plot(t, XP, label='XP', linewidth=2)
    ax[0].plot(t, XPE, label='XPE', linewidth=2)
    ax[0].plot(t, XPP, label='XPP', linewidth=2)
    ax[0].axvline(x=1, color='gray', linestyle='--', alpha=0.7, label='E step change')
    ax[0].set_title('Signal Transduction System Dynamics')
    ax[0].set_xlabel('Time')
    ax[0].set_ylabel('Concentration')
    ax[0].legend()
    ax[0].grid(True, alpha=0.3)
    
    # Plot E separately since it has different magnitude
    ax[1].plot(t, E, label='E', linewidth=2, color='green')
    ax[1].axvline(x=1, color='gray', linestyle='--', alpha=0.7)
    ax[1].set_title('Enzyme Concentration')
    ax[1].set_xlabel('Time')
    ax[1].set_ylabel('Concentration')
    ax[1].legend()
    ax[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    plt.close()
    # Print the final values
    print(f"Final values at t={t_max}:")
    print(f"X: {X[-1]:.4f}")
    print(f"XE: {XE[-1]:.4f}")
    print(f"XP: {XP[-1]:.4f}")
    print(f"XPE: {XPE[-1]:.4f}")
    print(f"XPP: {XPP[-1]:.4f}")
    print(f"E: {E_step:.4f}")  # E value after step change
    
    # Print mass conservation check
    total_initial = X_init + XE_init + XP_init + XPE_init + XPP_init
    total_final = X[-1] + XE[-1] + XP[-1] + XPE[-1] + XPP[-1]
    print(f"\nMass conservation check:")
    print(f"Total initial X species: {total_initial:.4f}")
    print(f"Total final X species: {total_final:.4f}")
    print(f"Difference: {total_final - total_initial:.8f}")

### Interactive cell

In [ ]:
# Create a more comprehensive interactive dashboard
def create_interactive_dashboard():
    # Initial condition widgets
    ic_widgets = [
        FloatSlider(min=0, max=50, step=0.5, value=28, description='X(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=1, step=0.01, value=0.1, description='XE(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=1, step=0.01, value=0.2, description='XP(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='XPE(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=0.1, step=0.001, value=0.02, description='XPP(0):', style={'description_width': '60px'})
    ]
    
    # Enzyme widgets
    enzyme_widgets = [
        FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='E(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=20, step=0.1, value=10, description='E step:', style={'description_width': '60px'})
    ]
    
    # Kinetic parameter widgets
    k_widgets = [
        FloatSlider(min=0.1, max=5, step=0.1, value=1, description='k1:', style={'description_width': '40px'}),
        FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='k2:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=10, step=0.1, value=4, description='k3:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=1, description='k4:', style={'description_width': '40px'}),
        FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='k5:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=10, step=0.1, value=4, description='k6:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=2, description='k7:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=2, description='k8:', style={'description_width': '40px'})
    ]
    
    # Time widgets
    time_widget = FloatSlider(min=5, max=50, step=1, value=10, description='Max Time:', style={'description_width': '80px'})
    
    # Output function for the interactive widgets
    out = interactive(plot_ode_solution, 
                      t_max=time_widget,
                      X_init=ic_widgets[0], XE_init=ic_widgets[1], XP_init=ic_widgets[2], 
                      XPE_init=ic_widgets[3], XPP_init=ic_widgets[4],
                      E_init=enzyme_widgets[0], E_step=enzyme_widgets[1],
                      k1=k_widgets[0], k2=k_widgets[1], k3=k_widgets[2], k4=k_widgets[3], 
                      k5=k_widgets[4], k6=k_widgets[5], k7=k_widgets[6], k8=k_widgets[7])
    out.update()
    # Create layout
    ic_box = VBox(ic_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    enzyme_box = VBox(enzyme_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    k_box = VBox(k_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    
    # Create a reset button
    reset_button = Button(description='Reset Parameters', button_style='warning')
    
    def reset_params(b):
        ic_widgets[0].value = 28
        ic_widgets[1].value = 0.1
        ic_widgets[2].value = 0.2
        ic_widgets[3].value = 0.01
        ic_widgets[4].value = 0.02
        enzyme_widgets[0].value = 0.01
        enzyme_widgets[1].value = 10
        k_widgets[0].value = 1
        k_widgets[1].value = 0.1
        k_widgets[2].value = 4
        k_widgets[3].value = 1
        k_widgets[4].value = 0.1
        k_widgets[5].value = 4
        k_widgets[6].value = 2
        k_widgets[7].value = 2
        time_widget.value = 10
    
    reset_button.on_click(reset_params)
    
    # Top row - time widget and reset button
    top_row = HBox([time_widget, reset_button], layout=Layout(padding='10px'))
    
    # Middle row - initial conditions and enzyme parameters
    middle_row = HBox([
        VBox([
            ic_box,
            enzyme_box
        ]),
        k_box
    ])
    
    # Combine everything
    dashboard = VBox([
        top_row,
        middle_row,
        out.children[-1]  # The output area
    ])
    
    return dashboard

In [ ]:
# Run the interactive dashboard
dashboard = create_interactive_dashboard()
display(dashboard)

## Write equations for diagrams - Signal transduction systems (Periodic enzyme signals)

### Define system of ODEs with periodic enzyme concentration

In [ ]:
def solve_signal_transduction_odes_fluctuating(
    t_max=10.0, 
    X_init=28, XE_init=0.1, XP_init=0.2, XPE_init=0.01, XPP_init=0.02,
    E_min=0.01, E_max=10, fluctuation_type='rectangle', frequency=1.0, decay_rate=0.1,
    k1=1, k2=0.1, k3=4, k4=1, k5=0.1, k6=4, k7=2, k8=2
):
    """
    Solves the system of ODEs for the signal transduction model with fluctuating E
    """
    
    def get_E_value(t):
        """Calculate E value at time t based on fluctuation type"""
        if fluctuation_type == 'rectangle':
            # Rectangle/square wave fluctuation
            E = E_max if np.sin(2 * np.pi * frequency * t) < 0 else E_min
        elif fluctuation_type == 'exponential_decay':
            # Exponential decay with oscillation
            E = E_min + (E_max - E_min) * np.exp(-decay_rate * t) * (0.5 + 0.5 * np.sin(2 * np.pi * frequency * t))
        else:
            raise ValueError(f"Unknown fluctuation_type: {fluctuation_type}")
        
        return E
    
    # Define the ODE system
    def ode_system(t, y):
        X, XE, XP, XPE, XPP = y
        
        # Calculate current E value
        E_current = get_E_value(t)
        
        # System of ODEs
        dXdt = -k1 * X * E_current + k2 * XE + k7 * XP
        dXEdt = k1 * X * E_current - (k2 + k3) * XE
        dXPdt = k3 * XE - k7 * XP - k4 * XP * E_current + k5 * XPE + k8 * XPP
        dXPEdt = k4 * XP * E_current - (k5 + k6) * XPE
        dXPPdt = k6 * XPE - k8 * XPP
        
        return [dXdt, dXEdt, dXPdt, dXPEdt, dXPPdt]
    
    # Initial conditions
    y0 = [X_init, XE_init, XP_init, XPE_init, XPP_init]
    
    # Time points
    t_span = (0, t_max)
    t_eval = np.linspace(0, t_max, 1000)
    
    # Solve the ODE system
    solution = solve_ivp(
        ode_system, 
        t_span, 
        y0, 
        method='RK45', 
        t_eval=t_eval,
        rtol=1e-6,
        atol=1e-8
    )
    
    return solution

### Plot functions

In [ ]:
# Create global output widget
out = Output()

def plot_solution(**kwargs):
    """Plot the solution with current parameters"""
    global out  # Use the global output widget
    
    with out:
        out.clear_output(wait=True)  # Clear previous output
        
        # Solve the system
        solution = solve_signal_transduction_odes_fluctuating(**kwargs)
        
        t = solution.t
        X, XE, XP, XPE, XPP = solution.y
        
        # Calculate E values for each time point
        E = np.zeros_like(t)
        for i, time in enumerate(t):
            if kwargs['fluctuation_type'] == 'rectangle':
                E[i] = kwargs['E_max'] if np.sin(2 * np.pi * kwargs['frequency'] * time) < 0 else kwargs['E_min']
            elif kwargs['fluctuation_type'] == 'exponential_decay':
                E[i] = kwargs['E_min'] + (kwargs['E_max'] - kwargs['E_min']) * np.exp(-kwargs['decay_rate'] * time) * (0.5 + 0.5 * np.sin(2 * np.pi * kwargs['frequency'] * time))
        
        # Close any existing matplotlib figures to prevent accumulation
        plt.close('all')
        
        # Create plots
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        
        # Plot species concentrations
        ax[0].plot(t, X, label='X', linewidth=2)
        ax[0].plot(t, XE, label='XE', linewidth=2)
        ax[0].plot(t, XP, label='XP', linewidth=2)
        ax[0].plot(t, XPE, label='XPE', linewidth=2)
        ax[0].plot(t, XPP, label='XPP', linewidth=2)
        ax[0].set_title(f'Signal Transduction with {kwargs["fluctuation_type"].title()} E', fontsize=14)
        ax[0].set_xlabel('Time', fontsize=12)
        ax[0].set_ylabel('Concentration', fontsize=12)
        ax[0].legend(fontsize=11)
        ax[0].grid(True, alpha=0.3)
        ax[0].set_ylim(bottom=0)
        
        # Plot enzyme concentration
        ax[1].plot(t, E, label=f'E ({kwargs["fluctuation_type"]})', linewidth=2, color='green')
        ax[1].axhline(y=kwargs['E_min'], color='red', linestyle='--', alpha=0.5, 
                      label=f'E_min = {kwargs["E_min"]}')
        ax[1].axhline(y=kwargs['E_max'], color='red', linestyle='--', alpha=0.5, 
                      label=f'E_max = {kwargs["E_max"]}')
        
        title_extra = f', Freq: {kwargs["frequency"]}'
        if kwargs['fluctuation_type'] == 'exponential_decay':
            title_extra += f', Decay: {kwargs["decay_rate"]}'
        ax[1].set_title(f'Enzyme Concentration{title_extra}', fontsize=14)
        ax[1].set_xlabel('Time', fontsize=12)
        ax[1].set_ylabel('Concentration', fontsize=12)
        ax[1].legend(fontsize=11)
        ax[1].grid(True, alpha=0.3)
        ax[1].set_ylim(bottom=0)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary
        print(f"\n{'='*50}")
        print(f"SIMULATION SUMMARY")
        print(f"{'='*50}")
        print(f"Final concentrations at t={kwargs['t_max']}:")
        print(f"  X: {X[-1]:.4f}     XE: {XE[-1]:.4f}     XP: {XP[-1]:.4f}")
        print(f"  XPE: {XPE[-1]:.4f}    XPP: {XPP[-1]:.4f}    E: {E[-1]:.4f}")

### Default params setting

In [ ]:
# Default values dictionary for reset functionality
default_values = {
    # Initial conditions
    'X_init': 28, 'XE_init': 0.1, 'XP_init': 0.2, 'XPE_init': 0.01, 'XPP_init': 0.02,
    # Enzyme parameters
    'E_min': 0.01, 'E_max': 10, 'fluctuation_type': 'rectangle', 'frequency': 1.0, 'decay_rate': 0.1,
    # Rate constants
    'k1': 1, 'k2': 0.1, 'k3': 4, 'k4': 1, 'k5': 0.1, 'k6': 4, 'k7': 2, 'k8': 2,
    # Time
    't_max': 10
}
# Configure slider style for better visibility
slider_style = {'description_width': '100px'}
slider_layout = Layout(width='400px')
# Initial condition widgets
widgets_dict = {
    # Initial conditions
    'X_init': FloatSlider(min=0, max=50, step=0.5, value=default_values['X_init'], 
                          description='X(0):', style=slider_style, layout=slider_layout),
    'XE_init': FloatSlider(min=0, max=1, step=0.01, value=default_values['XE_init'], 
                            description='XE(0):', style=slider_style, layout=slider_layout),
    'XP_init': FloatSlider(min=0, max=1, step=0.01, value=default_values['XP_init'], 
                            description='XP(0):', style=slider_style, layout=slider_layout),
    'XPE_init': FloatSlider(min=0, max=0.1, step=0.001, value=default_values['XPE_init'], 
                            description='XPE(0):', style=slider_style, layout=slider_layout),
    'XPP_init': FloatSlider(min=0, max=0.1, step=0.001, value=default_values['XPP_init'], 
                            description='XPP(0):', style=slider_style, layout=slider_layout),
    
    # Enzyme parameters
    'E_min': FloatSlider(min=0, max=0.5, step=0.01, value=default_values['E_min'], 
                          description='E min:', style=slider_style, layout=slider_layout),
    'E_max': FloatSlider(min=0, max=20, step=0.1, value=default_values['E_max'], 
                          description='E max:', style=slider_style, layout=slider_layout),
    'fluctuation_type': Dropdown(options=['rectangle', 'exponential_decay'], 
                                  value=default_values['fluctuation_type'], 
                                  description='Type:', style=slider_style, layout=slider_layout),
    'frequency': FloatSlider(min=0.1, max=5.0, step=0.1, value=default_values['frequency'], 
                              description='Frequency:', style=slider_style, layout=slider_layout),
    'decay_rate': FloatSlider(min=0.01, max=2.0, step=0.01, value=default_values['decay_rate'], 
                              description='Decay rate:', style=slider_style, layout=slider_layout),
    
    # Rate constants
    'k1': FloatSlider(min=0.1, max=5, step=0.1, value=default_values['k1'], 
                      description='k1:', style=slider_style, layout=slider_layout),
    'k2': FloatSlider(min=0.01, max=1, step=0.01, value=default_values['k2'], 
                      description='k2:', style=slider_style, layout=slider_layout),
    'k3': FloatSlider(min=0.1, max=10, step=0.1, value=default_values['k3'], 
                      description='k3:', style=slider_style, layout=slider_layout),
    'k4': FloatSlider(min=0.1, max=5, step=0.1, value=default_values['k4'], 
                      description='k4:', style=slider_style, layout=slider_layout),
    'k5': FloatSlider(min=0.01, max=1, step=0.01, value=default_values['k5'], 
                      description='k5:', style=slider_style, layout=slider_layout),
    'k6': FloatSlider(min=0.1, max=10, step=0.1, value=default_values['k6'], 
                      description='k6:', style=slider_style, layout=slider_layout),
    'k7': FloatSlider(min=0.1, max=5, step=0.1, value=default_values['k7'], 
                      description='k7:', style=slider_style, layout=slider_layout),
    'k8': FloatSlider(min=0.1, max=5, step=0.1, value=default_values['k8'], 
                      description='k8:', style=slider_style, layout=slider_layout),
    
    # Time
    't_max': FloatSlider(min=1, max=30, step=1, value=default_values['t_max'], 
                          description='T max:', style=slider_style, layout=slider_layout)
}

### Interactive cell

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import (FloatSlider, Dropdown, VBox, HBox, interactive, Layout, 
                       Button, HTML, Accordion, Tab, Output)
import ipywidgets as widgets

def create_interactive_dashboard():
    """
    Create an interactive dashboard with properly organized parameters and reset functionality
    """
    
    # Reset button
    reset_button = Button(description='Reset to Defaults', button_style='warning', 
                          layout=Layout(width='200px', height='35px'))
    
    def reset_parameters(b):
        """Reset all parameters to default values"""
        for key, value in default_values.items():
            widgets_dict[key].value = value
    
    reset_button.on_click(reset_parameters)
  
    # Create the interactive widget
    widget = interactive(plot_solution, **widgets_dict)
    
    # Organize widgets into tabs
    initial_conditions = VBox([
        HTML("<h3>Initial Conditions</h3>"),
        widgets_dict['X_init'],
        widgets_dict['XE_init'],
        widgets_dict['XP_init'],
        widgets_dict['XPE_init'],
        widgets_dict['XPP_init']
    ])
    
    enzyme_params = VBox([
        HTML("<h3>Enzyme Parameters</h3>"),
        widgets_dict['E_min'],
        widgets_dict['E_max'],
        widgets_dict['fluctuation_type'],
        widgets_dict['frequency'],
        widgets_dict['decay_rate']
    ])
    
    rate_constants = VBox([
        HTML("<h3>Rate Constants</h3>"),
        widgets_dict['k1'],
        widgets_dict['k2'],
        widgets_dict['k3'],
        widgets_dict['k4'],
        widgets_dict['k5'],
        widgets_dict['k6'],
        widgets_dict['k7'],
        widgets_dict['k8']
    ])
    
    simulation_params = VBox([
        HTML("<h3>Simulation Parameters</h3>"),
        widgets_dict['t_max'],
        HTML("<br>"),
        reset_button
    ])
    
    # Create tabs
    tab = Tab()
    tab.children = [initial_conditions, enzyme_params, rate_constants, simulation_params]
    tab.set_title(0, 'Initial Conditions')
    tab.set_title(1, 'Enzyme Parameters')
    tab.set_title(2, 'Rate Constants')
    tab.set_title(3, 'Simulation')
    
    # Display everything
    display(VBox([tab, out]))
    
    # Add observers to update plot when parameters change
    for widget in widgets_dict.values():
        widget.observe(lambda x: plot_solution(**{k: v.value for k, v in widgets_dict.items()}), 'value')
    
    # Initial plot
    plot_solution(**{k: v.value for k, v in widgets_dict.items()})
    
    return widget

In [ ]:
create_interactive_dashboard();